# Purpose

The purpose of this script is to analyze stock following principals outlines by Benjamin Graham. These principles involve the following guidlines

- PE ratio below 9 but positivie
- Current Ratio (Current Assets/Current Liab) greater than 1.5
- Debt Load (Total Debt/ Current Assets) Less than or equal to 1.1
- Positive EPS for each quarter in the last year
- Price To Book ratio less than or equal to 1.2 
- Look for dividends

Additionally I will be calculating the following variables for my own interest
- NCAV (Net Current Assets Value) = (Net Assets-TotalLiabilities-preferred stock)

## Load Name of Stocks
The section uses the yahoo finance library to get the names of many stocks. 

In [1]:
import yahoo_fin.stock_info as si
dow_list = si.tickers_dow() #Loads Tickers in dow jones
nasdaq_list = si.tickers_nasdaq() #Loads Tickers in NASDAQ
sp500_list = si.tickers_sp500()
other_list = si.tickers_other() #Loads Tickers in Other Indexes


## Load yfinance and Create Functions To Read Data I am interested In 
The yfinance library is what will primarily be used to load information for each of the stock tickers. Within this libary 3 primary functions will be called they are   

- stk.info #(used for PE, EPS, Price to Book values)
- stk.balance_sheet (used for Current Ratio, Debt Load, NCAV)
- stk.dividends (used for dividend calculations

In order to use these functions more efficiently they will be integrated into a number of my own creation. The first of these functions is below and is used to read useful information from the stk.info call

In [71]:
import yfinance as yf
import numpy
import pandas as pd

def extractInfo(Ticker):
    commonStock = False
    divRate = False
    forPE = 0
    trailPE = 0
    forEPS = 0
    trailEPS = 0
    P2B = 0
    
    #Ticers with these values in their name were often for funds preferred stock etc
    #So this line removes those other investment mediums
    if '$' in Ticker or '.' in Ticker:
        return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B
    
    stk = yf.Ticker(Ticker) #Creates object to load stock data
    stkInfo = stk.info #Loads basic stock info 
    
    #In yfinance all common stock have forwardPE listed but funds and other investments do not.
    #So this line removes those other investment mediums
    if 'forwardPE' not in stkInfo or stkInfo['forwardPE'] == None: 
        return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B
    
    commonStock = True
    
    #Reads stock info where it is available to be read
    divRate = stkInfo['dividendRate'] if 'dividendRate' in stkInfo else 'None'
    forPE = stkInfo['forwardPE']
    trailPE = stkInfo['trailingPE'] if 'trailingPE' in stkInfo else 'Negative'
    forEPS = stkInfo['forwardEps'] if 'forwardEps' in stkInfo else 'None'
    trailEPS = stkInfo['trailingEps'] if 'trailingEps' in stkInfo else 'None'
    P2B = stkInfo['priceToBook'] if 'priceToBook' in stkInfo else 'None'
    
    return commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk

The second function will be responsible for reading the balance sheet for the stk. It is assumed here that the function extractInfo has already been read ahead of this function and has prevented any non-common-stock investments from progressing further into the data extraction process 

In [72]:
def extractBalSheet(stk):
    stkBal= stk.balancesheet #Returns last four years of annual balance sheet
    recBal = stkBal[stkBal.columns[0]] #Returns the most recent balance sheet
    
    curAsts = recBal['Total Current Assets'] if 'Total Current Assets' in recBal else 'None'
    curLiabs = recBal['Total Current Liabilities'] if 'Total Current Liabilities' in recBal else 'None'
    totAsts = recBal['Total Assets'] if 'Total Assets' in recBal else 'None'
    totLiabs = recBal['Total Liab'] if 'Total Liab' in recBal else 'None'
    
    stkHoldEqty = totAsts-totLiabs if (type(totAsts)!=str and type(totLiabs)!=str) else 'None'
    curRat = curAsts/curLiabs if (type(curAsts)!=str and type(curLiabs)!=str and curLiabs!=0) else 'None'
    debtLoad = totLiabs/curAsts if (type(curAsts)!=str and type(totLiabs)!=str and curAsts!=0) else 'None'
    
    return stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs
    

The third function is responsible for reading the earnings of the data for the last four monthes.

In [73]:
def extractPrev4QuartEarnings(stk):
    stkQuartEarn = stk.quarterly_earnings
    earn = stkQuartEarn['Earnings'] if 'Earnings' in stkQuartEarn.columns else 'None'
    numQ = earn.shape[0]
    mostRec = earn[-1] if numQ >= 1 else 'None' #Most recent is last value
    secMostRec = earn[-2] if numQ >= 2 else 'None'
    thrdMostRec = earn[-3] if numQ >= 3 else 'None'
    frthMostRec = earn[-4] if numQ >= 4 else 'None'
    
    return mostRec, secMostRec, thrdMostRec, frthMostRec
    

## Loop Through Stocks in the Dow

In [79]:
stkDF = pd.DataFrame(columns = ['Ticker', 'divRate','forward PE','Trail PE','Forward EPS','Trail EPS','Price2Book',
                            'stock Holder Equity','Current Ration','Debt Load','Current Assets',
                            'Current Liabillities','Total Assets','Total Liabilities',
                            'Most Recent Quarter Earnings','2nd MRE','3rd MRE','4th MRE'])

for Ticker in dow_list:
    commonStock,divRate,forPE,trailPE,forEPS,trailEPS,P2B,stk = extractInfo(Ticker)
    
    if commonStock == True:
        stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs = extractBalSheet(stk)
        mostRec, secMostRec, thrdMostRec, frthMostRec = extractPrev4QuartEarnings(stk)
        
        nr = [Ticker,divRate,forPE,trailPE,forEPS,trailEPS,P2B,
             stkHoldEqty,curRat,debtLoad,curAsts,curLiabs,totAsts,totLiabs,
             mostRec, secMostRec, thrdMostRec, frthMostRec]
        
        stkDF = stkDF.append(pd.Series(nr,
                                      index = stkDF.columns),
                            ignore_index=True)
        
        
        

In [80]:
print(stkDF)

   Ticker divRate  forward PE  Trail PE  Forward EPS  Trail EPS Price2Book  \
0    AAPL    0.82   28.725174   37.9207         4.33      3.280    32.3149   
1    AMGN     6.4   13.459142   18.4644        17.01     12.399      12.19   
2     AXP    1.72   18.245200   30.4087         6.77      4.062    4.54686   
3      BA    None  171.427540  Negative         1.38     -7.887       None   
4     CAT    4.12   24.233060   29.6485         7.38      6.032    6.49925   
5     CRM    None   64.917380   59.0617         3.51      3.858    5.16654   
6    CSCO    1.44   13.166172   17.9636         3.37      2.470    4.90927   
7     CVX    5.16   34.543396  Negative         2.65     -6.182    1.29719   
8     DIS    None   31.958420  Negative         4.81     -1.588    3.32886   
9     DOW     2.8   20.055147  Negative         2.72     -3.167    3.27529   
10     GS       5    9.898052   13.7684        24.13     17.347    1.05208   
11     HD       6   21.140549   22.6382        12.38     11.561 

## Filters Stocks Based on Benjamin Graham Requirements